In [1]:
"Set working directory"
import os
os.chdir('../')

In [2]:
import json
import numpy as np
import os
import pandas as pd
import networkx as nx

from math import exp, log
from typing import List, Tuple
from tqdm import tqdm

from socpd.objects import Object
from socpd.sequences import AgentList
#agentpy
from socpd.model import Model
from socpd.network import Network

from socpd.agent_nw import Individual, Populating
from socpd.hypothesis_nw import Hypothesis

In [3]:

class SocPD(Model, Hypothesis):  
    def setup(self) :
               
        # Set-up Hypothesis 
        Hypo_dict = self.p.Hypothesis_settings #_____________ added
        Hypothesis.validate_n_read_hypotheses(Hypo_dict)#__________________added

        #call-out hypothesis on actions
        self._dir_params = Hypothesis.dir_params
        self.status_var = Hypothesis.status_var
        #self.env_beta = Hypothesis.env_beta
        self.all_possible_features = Hypothesis.all_possible_features
        
        self.self_adopt = Hypothesis.self_adopt
        self.homo_neg_adopt = Hypothesis.homo_neg_adopt
        self.homo_pos_adopt = Hypothesis.homo_pos_adopt     
        self.rules = Hypothesis.rules

         # Private variables
        self.status_quo : float = .0
        self._use_ipf : bool = None
        pop = self.pop = self.p['pop']
        den = self.p['den']
        s = int(np.ceil(np.sqrt(pop/den)))
        self.report('grid_size', s)

        # Update default values with input value
        ###### seed, pop, use_ipf, _steps, grid_size (by density)
        if 'env_beta' in self.p:
            self._env_beta = float(self.p['env_beta'])
        else:
            self._env_beta = 0.0
        self.report("intervention effect-env_beta", self._env_beta)
        
        if 'use_ipf' in self.p:
            self._use_ipf = self.p['use_ipf']
        else:
            self._use_ipf = False 
        self.report('use_ipf', self._use_ipf)
                #----------------------------------------------#
        #_______GRID_____________________________________________
        # setup model's Grid
        self.grid = Grid(self, (s, s), track_empty=True)

        #_______AGENTS___________________________________________
        #__generate agents
        self.agents = AgentList(self, pop, Individual)
        #__generate features then update agent's features and status
        self.Populating = Populating(self)
        if self._use_ipf:
            _feature_iter = self.Populating.populate_ipf(self._dir_params, pop)
        else:
            _feature_iter = self.Populating.populate_simple(self._dir_params, pop)
        #__update agent's features 
        for i, a in enumerate(self.agents):
            a.features = _feature_iter[i]
        #for a, fs in zip (self.agents, _feature_iter):
        #    a.features  =  fs
        #__update Agent status based on feature generation        
        self.agents.get_status_step0()
        
        #_______ADD AGENTS ON THE GRID_________________________
        self.grid.add_agents(self.agents, random=True, empty=True)


    def update(self): 
        """agents = [a for a in self.agents]
        print('env_beta',agents[0].env_beta)
        print('\n\nparams_self\n',agents[0].params_self)
        print('\n\nparams_nw actions\n', agents[0].params_nw.index)
        print('\n\nmasked_self_adopt\n',agents[0].masked_self_adopt)
        print('\n\nnegative nw-actions to adopt:\n',agents[0].masked_neg)
        print('\n\npositive nw-actions to adopt:\n',agents[0].masked_pos)
        print('\n\nall actions to adopt:\n',agents[0].masked_nw_adopt)
        print('\n\nstatus a0', agents[0].status)
        print('\n\nfeatures a0', agents[0].features)
        print('\n\nstatus a1', agents[5].status)
        print('\n\nfeatures a1', agents[5].features)"""
        agents = [a for a in self.agents]
        n_neigh = [len([n for n in a.grid.neighbors(a)]) for a in agents]
        print(max(n_neigh))

In [3]:
class SocPD(Model, Hypothesis):  
    def setup(self) :
            
        # Set-up Hypothesis 
        Hypo_dict = self.p.Hypothesis_settings #_____________ added
        Hypothesis.validate_n_read_hypotheses(Hypo_dict)#__________________added

        #call-out hypothesis on actions
        self._dir_params = Hypothesis.dir_params
        self.status_var = Hypothesis.status_var
        self.all_possible_features = Hypothesis.all_possible_features

        self.homo_neg = Hypothesis.homo_neg
        self.homo_pos = Hypothesis.homo_pos    
        self.rules = Hypothesis.rules
        
        # Private attributes_________________________________________________________________
            # status_quo, env_beta,use_ipf, pop, 
        self.status_quo : float = .0
        if 'env_beta' in self.p:
            self._env_beta = float(self.p['env_beta'])
        else:
            self._env_beta = 0.0
        self.report("intervention's effect-env_beta", self._env_beta)

            # Specifying use ipf
        #self._use_ipf : bool = None
        if 'use_ipf' in self.p:
            self._use_ipf = self.p['use_ipf']
        else:
            self._use_ipf = False 
        self.report('use_ipf', self._use_ipf)
                
        #----------------------------------------------#
        #_______PREPARE NETWORK_____________________

        pop = self.pop = self.p['pop']
        m = self.p['m']
        p = self.p['p']
        q = self.p['q']
        graph = nx.extended_barabasi_albert_graph(
            n = pop, 
            m = m, # there are 1-p-q chance a new nodes can connect with m other existing nodes based on attachment preference
            p = p, # each existing nodes has p prob to form a new links to the others with attachment preference
            q = q, # each existing nodes has q prob to rewire one of thier existin links with attachment preference
            seed=self.random)
        self.network = Network(self, graph)
        
            # Report network's attributes
        degree_sequence = sorted((d for n, d in graph.degree()), reverse=True)
        self.report("Max/Min nw-size", f"{max(degree_sequence)}, {min(degree_sequence)}")
        
        
        #_______GENERATE AGENTS______________________
        self.agents = AgentList(self, pop, Individual)
        
            # generate features then update agent's features and status
            # Specifying use_ipf
        self.Populating = Populating(self)
        if self._use_ipf:
            _feature_iter = self.Populating.populate_ipf(self._dir_params, pop)
        else:
            _feature_iter = self.Populating.populate_simple(self._dir_params, pop)
            
            # update agent's features and status step 0
        for i, a in enumerate(self.agents):
            a.features = _feature_iter[i]
        self.agents.get_status_step0()
        
        #_______ADD AGENTS ON network_________________________
        
        self.network.add_agents(self.agents, self.network.nodes)

    def update(self): 
        positive_p = len(self.agents.select(self.agents.status==True))/self.pop
       
        self.status_quo = log(positive_p/(1-positive_p))
        print(self.status_quo)
        
        # Stop simulation if all are positive or negative
        if positive_p == 0 or positive_p == 1:
            self.stop()

In [4]:
dir_params = 'parameters_demo'
all_possible_actions=  [ 'Veg_prob',
                          'concerns_health',
                         'concerns_animal_welfare',
                         'concerns_environement',
                         'concerns_convinience',
                         'concerns_familiarity',
                         'concerns_nutritional_adeq',
                         'concerns_taste',
                         'concerns_price',
                         'concerns_hunger',
                         'homo_veg_inf',
                         'homo_om_inf',

                        ]

Hypothesis_settings = { 'dir_params' : dir_params , # folder name stored actions parameters
                        'status_var':'Vegetarian',  # the variables deciding agents' status
                        # all actions from the actions parameters
                        'all_possible_actions':all_possible_actions,
                        # all actions/events influencing directly on the agent
                        'actions_to_self':['Veg_prob',
                                            'concerns_health',
                                            'concerns_animal_welfare',
                                            'concerns_environement',
                                            'concerns_convinience',
                                            'concerns_familiarity',
                                            'concerns_nutritional_adeq',
                                            'concerns_taste',
                                            'concerns_price',
                                            'concerns_hunger',],
                        # all actions/events from the agent influencing on others in thier networks
                        'actions_to_nw':['homo_veg_inf',
                                        'homo_om_inf',],

                        # actions/events from the agents having negative effects on thier networks 
                        'actions_to_nw_neg':['homo_om_inf'],
                        # actions/events from the agents having positive effects on thier networks 
                        'actions_to_nw_pos':['homo_veg_inf',],
                        }
 


parameters = {'Hypothesis_settings' : Hypothesis_settings,
            'steps': 5,    
            'seed' : 12345,   
            'pop' : 1000, # size of the population
            'm': 1, # Number of edges with which a new node attaches to existing nodes
            'p' : .05, # each existing nodes has p prob to form m new link(s) to the others with attachment preference
            'q' : .5, # each existing nodes has q prob to rewire m existing link(s) with attachment preference
            #'use_ipf': False,   
            } 

parameters_cluster = {'Hypothesis_settings' : Hypothesis_settings,
            'steps': 3,    
            'seed' : 12345,   
            'pop' : 1000,
            'm': 1, 
            'p' : .5
            } 

In [5]:
model = SocPD(parameters)
model.run()

Populating individuals: 100%|██████████| 1000/1000 [00:00<00:00, 13728.68i/s]

-0.8808580568795056
-0.8808580568795056
Completed: 1 steps-0.8808580568795056
Completed: 2 steps-0.8808580568795056
Completed: 3 steps-0.8808580568795056
Completed: 4 steps-0.8808580568795056
Completed: 5 steps
Run time: 0:00:00.727322
Simulation finished


DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 7 keys
'reporters': DataFrame with 4 variables and 1 row
}